In [1]:
import os
import pandas as pd
import string

In [2]:
f = pd.read_csv('../astronomy - Sheet.csv')

In [3]:
f

,Unnamed: 0,type,key,freq
8201,T,n,star,6455
156,T,n,telescope,3893
8416,T,n,earth,2112
20252,T,n,light,2054
3394,T,n,moon,1893
...,...,...,...,...
6576,T,nn,faculae,10
2345,T,n,flare star,10
1237,T,aan,geosynchronous orbit,10
47,T,aaa,hydrostatic equilibrium,10


In [4]:
terms = list(f['key'].values)
# terms

In [5]:
f.set_index('key', inplace = True)


In [6]:
# f.loc['dark sky']['Unnamed: 0'][0]
'star' in f.index

True

In [7]:
import spacy
from spacy.matcher import PhraseMatcher

In [8]:
nlp = spacy.load('en_core_web_sm')
# matched_sents = []  # Collect data of matched sentences to be visualized

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    sent = span.sent  # Sentence containing matched span
    # Append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    # print (span)
    # print (f.loc[str(span).lower()]['Unnamed: 0'][0])
    modified_span = str(span).lower().translate(str.maketrans('', '',string.punctuation))
    if modified_span in f.index:
        match_ents = [{
            "start": span.start_char - sent.start_char,
            "end": span.end_char - sent.start_char,
            # "label": "AstroTerm",
        
            "label": f.loc[modified_span]['Unnamed: 0'][0],
        }]
    else:
        match_ents = [{
            "start": span.start_char - sent.start_char,
            "end": span.end_char - sent.start_char,
            # "label": "AstroTerm",
        
            "label": 'anonymous',
        }]
    matched_sents.append({"text": sent.text, "ents": match_ents})



In [9]:
matcher = PhraseMatcher(nlp.vocab)

patterns = [nlp.make_doc(text) for text in terms]

matcher.add("TerminologyList", collect_sents, *patterns)

In [10]:
test_file  = open("../corpus/Astromony_04.txt", encoding='utf-8')
text = test_file.read()
# text

In [17]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
nlp = English()
ruler = EntityRuler(nlp)
print("Match_By_Entity Ruler============================")
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)
doc = nlp(text)
# patterns
# matched_sents

Match_By_Entity Ruler============================


TypeError: '<' not supported between instances of 'str' and 'int'

In [11]:
# text = open("../corpus/Astromony_04.txt", encoding='utf-8')
# text = text.read()


In [12]:
# doc = nlp(text)
# matches = matcher(doc)
# for match_id, start, end in matches:
#     span = doc[start:end]
# #     print(span.text)

In [13]:
# span
nlp.max_length = 70000000000

In [14]:
from spacy import displacy
# displacy.render(matched_sents, style="ent", manual=True)


In [15]:
import glob

doc_files = []

path = '../corpus'
read_files = glob.glob( path + "/*.txt")

# with open( path + "/result.txt", "wb") as outfile:
#     for f in read_files:
#         macthed_sents = []
#         with open(f, "rb") as infile:
#             outfile.write(infile.read())
for file_name in read_files:
    matched_sents = []
    print(file_name)
    with open(file_name, encoding='utf-8') as text:
        text = text.read()
        doc = nlp(text)
        matches = matcher(doc)
        doc_files.extend(matched_sents)
    # break
        

../corpus\Astromony_04.txt
../corpus\Astromony_05.txt
../corpus\Astromony_06.txt
../corpus\Astromony_07.txt
../corpus\Astromony_08.txt
../corpus\Astromony_09.txt
../corpus\Astromony_11.txt
../corpus\Astromony_14.txt
../corpus\Astromony_15.txt
../corpus\Astromony_17.txt
../corpus\Astromony_18.txt
../corpus\Astromony_20.txt
../corpus\Astromony_22.txt
../corpus\Astromony_23.txt
../corpus\Astromony_24.txt
../corpus\Astromony_26.txt
../corpus\Astromony_27.txt
../corpus\Astromony_28.txt
../corpus\Astromony_29.txt
../corpus\Astromony_30.txt
../corpus\Astromony_31.txt
../corpus\Astromony_32.txt
../corpus\Astromony_35.txt
../corpus\Astromony_39.txt
../corpus\Astromony_40.txt
../corpus\Astromony_42.txt
../corpus\Astromony_43.txt
../corpus\Astromony_45.txt
../corpus\Astromony_47.txt
../corpus\Astromony_51.txt
../corpus\Astromony_55.txt
../corpus\Astromony_56.txt
../corpus\Astromony_57.txt
../corpus\Astromony_58.txt
../corpus\Astromony_59.txt
../corpus\Astromony_60.txt
../corpus\Astromony_61.txt
.

In [ ]:
# doc_files

In [ ]:
# matched_sents[:4]
# file  = open(path + '/result.txt', encoding='utf-8')
# text = file.read()
# text

# matched_sents = [] 

In [ ]:
# nlp.max_length = 70000000000
# doc = nlp(text)
# # create the PhraseMatcher object
# matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

In [19]:
# creating docs with files
TRAIN_DATA = {}
# LABEL = 'AstroTerm'
dic_text_previous = ""
inner_list = []
ent_num = 0
for dic in doc_files:
    
    dict_item = {}
    
    if dic['text'] == dic_text_previous:
        ent_num += 1
        inner_list.append((dic['ents'][0]['start'],dic['ents'][0]['end'],dic['ents'][0]['label']+str(ent_num)))
    else:
        ent_num = 0
        inner_list = []
        inner_list.append((dic['ents'][0]['start'],dic['ents'][0]['end'],dic['ents'][0]['label']))
    dic_text_previous = dic['text']
    dict_item['entities'] = inner_list
    TRAIN_DATA[dic['text']] = dict_item


TRAIN_DATA = [(k,v) for k,v in TRAIN_DATA.items()]

In [22]:
# TRAIN_DATA = [(k,v) for k,v in TRAIN_DATA.items()]
TRAIN_DATA
import json

with open('train_data.json', 'w') as f:
    json.dump(TRAIN_DATA, f)

In [21]:
# doc_files[0]
TRAIN_DATA[0]

("As usual, we've been deprived of clear sky viewing opportunities this winter and spring—all the more reason to start planning for the regional dark sky star parties.",
 {'entities': [(143, 151, 'T'), (152, 156, 'T1')]})

In [ ]:
# # preprocess training data
# TRAIN_DATA = []
# LABEL = 'AstroTerm'
# for dic in matched_sents:
#     inner_list = []
#     dict_item = {}
#     inner_list.append((dic['ents'][0]['start'],dic['ents'][0]['end'],LABEL))
#     dict_item['entities'] = inner_list
#     TRAIN_DATA.append((dic['text'],dict_item,))

In [111]:
nlp = spacy.blank("en")
n_iter = 100

In [112]:
from __future__ import unicode_literals, print_function

import plac
import random
import warnings
from pathlib import Path
# import spacy
from spacy.util import minibatch, compounding

In [113]:
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    
    
ner = nlp.get_pipe("ner")

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [115]:
# get names of other pipes to disable them during training
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
# only train NER
with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
    # show warnings for misaligned entity spans once
    warnings.filterwarnings("once", category=UserWarning, module='spacy')
    
    nlp.begin_training()

    for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.25,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)


c:\Users\ramee\Documents\M2-NLP-UnivL\Codes\univL\lib\site-packages\spacy\language.py:635: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  proc.begin_training(


ValueError: [E103] Trying to set conflicting doc.ents: '(62, 68, 'T2')' and '(62, 75, 'T3')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.

In [ ]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


In [36]:
output_dir = 'output_dir'
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to output_dir


In [40]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])



), ('find', '', 2), ('the', '', 2), ('ratio', '', 2), ('of', '', 2), ('the', '', 2), ('distances', '', 2), ('of', '', 2), ('the', '', 2), ('sun', '', 2), ('and', '', 2), ('the', '', 2), ('moon', 'AstroTerm', 3), ('by', '', 2), ('geometrical', '', 2), ('analysis', '', 2), ('.', '', 2)]
Entities [('dark sky', 'AstroTerm')]
Tokens [('The', '', 2), ('Perseids', '', 2), ('are', '', 2), ('reliable', '', 2), (',', '', 2), ('better', '', 2), ('of', '', 2), ('course', '', 2), ('from', '', 2), ('a', '', 2), ('dark', 'AstroTerm', 3), ('sky', 'AstroTerm', 1), ('site', '', 2), ('like', '', 2), ('the', '', 2), ('Oregon', '', 2), ('Star', '', 2), ('Party', '', 2), (',', '', 2), ('but', '', 2), ('visible', '', 2), ('from', '', 2), ('local', '', 2), ('venues', '', 2), ('as', '', 2), ('well', '', 2), ('.', '', 2)]
Entities []
Tokens [('(', '', 2), ('That', '', 2), ("'s", '', 2), ('why', '', 2), ('the', '', 2), ('moon', '', 2), ('just', '', 2), ('covers', '', 2), ('the', '', 2), ('sun', '', 2), ('during'

In [80]:
# with open('../corpus/test/test1.txt', 'r') as f:
#     lines = f.readlines()
# test = ' '.join(lines)

test = open('../corpus/Astromony_04.txt',encoding='utf-8')
test = test.read()
doc = nlp2(test)
# test
# doc

In [81]:
displacy.render(doc, style="ent")
# test.read()
# [(t.text, t.ent_type_, t.ent_iob) for t in doc]

In [74]:
# matches_test = matcher (doc)

AttributeError: 'NoneType' object has no attribute 'start_char'